In [ ]:
%cd /sfs/weka/scratch/axu5pa/DS_4002_Project_3

In [ ]:
pip install openpyxl

In [ ]:
import pandas as pd

# Load the Excel file into a DataFrame
# Replace 'PH2_dataset.xlsx' with your actual file path
df = pd.read_excel('/sfs/weka/scratch/axu5pa/DS_4002_Project_3/PH2Dataset/PH2_dataset.xlsx', skiprows=12)  # Adjust 'skiprows' based on where your actual data starts

# Display the first few rows to verify
print(df.head())

In [ ]:
from IPython.display import display

# Use display instead of print for a cleaner output in Jupyter Notebook
display(df)

In [ ]:
pip install pillow

In [ ]:
import os

# Change directory to the desired folder
os.chdir('/sfs/weka/scratch/axu5pa/DS_4002_Project_3')

# Verify if the current working directory has changed
print("Current Working Directory:", os.getcwd())

In [ ]:
import os

print("Current Working Directory:", os.getcwd())

In [ ]:
import os
import pandas as pd
from PIL import Image

# Load your dataframe (assuming it's already loaded, or you can load it from a CSV)
# df = pd.read_csv('your_dataframe.csv')  # Load your dataframe if needed

# Define the directory where your .bmp images are stored
image_dir = '/sfs/weka/scratch/axu5pa/DS_4002_Project_3/PH2Dataset/Images'

# List all .bmp files in the directory
image_files = [f for f in os.listdir(image_dir) if f.endswith('.bmp')]

# Create a dictionary to map image names (without .bmp) to file paths
image_dict = {os.path.splitext(f)[0]: os.path.join(image_dir, f) for f in image_files}

# Add a new column to the dataframe for image paths
df['Image Path'] = df['Image Name'].map(image_dict)

# Define a function to classify based on the 'X' values in the relevant columns
def classify_lesion(row):
    if row['Common Nevus'] == 'X':
        return 'Benign'
    elif row['Atypical Nevus'] == 'X':
        return 'Benign*'
    elif row['Melanoma'] == 'X':
        return 'Malignant'
    else:
        return 'Unknown'  # If none of the columns have an 'X'

# Apply the function to each row in the dataframe to create a new 'Class' column
df['Class'] = df.apply(classify_lesion, axis=1)

# Check if there are any unknown classifications
unknown_classifications = df[df['Class'] == 'Unknown']
if not unknown_classifications.empty:
    print("Some images were not classified correctly:")
    print(unknown_classifications)
else:
    print("All images classified successfully.")

# Drop the 'Lesion Type' column if it exists
if 'Lesion Type' in df.columns:
    df = df.drop(columns=['Lesion Type'])

# Check if there are any missing matches
missing_images = df[df['Image Path'].isnull()]

if not missing_images.empty:
    print("Some images were not found:")
    print(missing_images)
else:
    print("All images matched successfully.")

# Example: Open and display an image from the dataframe using PIL
for index, row in df.iterrows():
    img_path = row['Image Path']
    if img_path:  # Ensure there's a valid path
        img = Image.open(img_path)
        display(img)  # Display image in Jupyter Notebook (optional)

In [ ]:
from IPython.display import display

# Use display instead of print for a cleaner output in Jupyter Notebook
display(df)

In [ ]:
import pandas as pd

# Assuming df already has columns 'Common Nevus', 'Atypical Nevus', and 'Melanoma'

# Define a function to classify based on the 'X' values in the relevant columns
def classify_lesion(row):
    if row['Common Nevus'] == 'X':
        return 'Benign'
    elif row['Atypical Nevus'] == 'X':
        return 'Benign*'
    elif row['Melanoma'] == 'X':
        return 'Malignant'
    else:
        return 'Unknown'  # If none of the columns have an 'X'

# Apply the function to each row in the dataframe to create a new 'Class' column
df['Class'] = df.apply(classify_lesion, axis=1)

# Check if there are any unknown classifications
unknown_classifications = df[df['Class'] == 'Unknown']
if not unknown_classifications.empty:
    print("Some images were not classified correctly:")
    print(unknown_classifications)
else:
    print("All images classified successfully.")

# Drop the 'Lesion Type' column if it exists
if 'Lesion Type' in df.columns:
    df = df.drop(columns=['Lesion Type'])

# Display the first few rows of the dataframe to verify
from IPython.display import display

# Use display instead of print for a cleaner output in Jupyter Notebook
display(df)



In [ ]:
######## EDA Section ########

In [ ]:
pip install matplotlib seaborn pillow numpy opencv-python

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageStat
import numpy as np
import cv2

# Assuming df is already loaded with the 'Image Path' and 'Class' columns

# For reference:
benign_classes = ['Common Nevus']
benign_star_classes = ['Atypical Nevus']
malignant_classes = ['Melanoma']


In [ ]:
# Question 1: Are the classes balanced, or is there class imbalance?
# --------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the distribution of classes ('Benign', 'Benign*', 'Malignant')
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='Class', palette='Set2')

# Set plot title and labels
plt.title('Distribution of Benign, Benign*, and Malignant Lesions')
plt.xlabel('Class')
plt.ylabel('Count')

# Show the plot
plt.show()

# Print class distribution for severity check (percentage)
class_distribution = df['Class'].value_counts(normalize=True) * 100
print("Class Distribution (%):\n", class_distribution)

In [ ]:
# Question 2: Are the images consistent in resolution? Do they have similar aspect ratios?
# --------------------------------------------------------------
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Function to get image dimensions and aspect ratio
def get_image_dimensions(image_path):
    # Check if the image path is valid and not NaN
    if isinstance(image_path, str) and os.path.exists(image_path):
        with Image.open(image_path) as img:
            width, height = img.size
            aspect_ratio = width / height
            return width, height, aspect_ratio
    else:
        return None, None, None  # Return None for invalid paths

# Apply function to each row in dataframe and store results in new columns
df[['Width', 'Height', 'Aspect Ratio']] = df['Image Path'].apply(lambda x: pd.Series(get_image_dimensions(x)))

# Drop rows where dimensions couldn't be calculated (i.e., where Width is None)
df_cleaned = df.dropna(subset=['Width'])

# Plot histograms of image widths, heights, and aspect ratios
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.histplot(df_cleaned['Width'], bins=20, ax=axes[0], color='skyblue')
axes[0].set_title('Image Width Distribution')

sns.histplot(df_cleaned['Height'], bins=20, ax=axes[1], color='lightgreen')
axes[1].set_title('Image Height Distribution')

sns.histplot(df_cleaned['Aspect Ratio'], bins=20, ax=axes[2], color='salmon')
axes[2].set_title('Image Aspect Ratio Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate the average (mean) of 'Width', 'Height', and 'Aspect Ratio'
average_width = df_cleaned['Width'].mean()
average_height = df_cleaned['Height'].mean()
average_aspect_ratio = df_cleaned['Aspect Ratio'].mean()

# Print the results
print(f"Average Width: {average_width:.2f} pixels")
print(f"Average Height: {average_height:.2f} pixels")
print(f"Average Aspect Ratio: {average_aspect_ratio:.2f}")

In [ ]:
# Question 3: Is there a difference in color distribution across classes?
# --------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Convert 'X' values in color columns to binary (1 for 'X', 0 for NaN)
color_columns = ['White', 'Red', 'Light-Brown', 'Dark-Brown', 'Blue-Gray', 'Black']
df[color_columns] = df[color_columns].applymap(lambda x: 1 if x == 'X' else 0)

# Melt the dataframe to long format for easier plotting
df_melted = df.melt(id_vars='Class', value_vars=color_columns, var_name='Color', value_name='Presence')

# Plot the distribution of colors by class
plt.figure(figsize=(10, 6))
sns.barplot(data=df_melted, x='Color', y='Presence', hue='Class', ci=None, palette='Set2')

# Set plot title and labels
plt.title('Color Distribution Across Lesion Classes')
plt.xlabel('Color')
plt.ylabel('Average Presence (1 = Present, 0 = Absent)')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Question 4: Do images differ significantly in brightness or contrast?
# --------------------------------------------------------------
import pandas as pd
from PIL import Image, ImageStat
import matplotlib.pyplot as plt
import seaborn as sns

# Function to calculate brightness and contrast of an image
def get_brightness_contrast(image_path):
    # Check if the image path is valid and not NaN
    if isinstance(image_path, str) and os.path.exists(image_path):
        img = Image.open(image_path).convert('L')  # Convert to grayscale for brightness/contrast calculation
        stat = ImageStat.Stat(img)
        
        # Brightness: Mean pixel value in grayscale image
        brightness = stat.mean[0]
        
        # Contrast: Standard deviation of pixel values (higher std -> higher contrast)
        contrast = stat.stddev[0]
        
        return brightness, contrast
    else:
        return None, None  # Return None for invalid paths

# Apply function to each row in dataframe and store results in new columns
df[['Brightness', 'Contrast']] = df['Image Path'].apply(lambda x: pd.Series(get_brightness_contrast(x)))

# Drop rows where brightness or contrast couldn't be calculated (i.e., where Brightness is None)
df_cleaned = df.dropna(subset=['Brightness'])

# Plot histograms of brightness and contrast values across images
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

sns.histplot(df_cleaned['Brightness'], bins=20, ax=axes[0], color='orange')
axes[0].set_title('Brightness Distribution')

sns.histplot(df_cleaned['Contrast'], bins=20, ax=axes[1], color='purple')
axes[1].set_title('Contrast Distribution')

plt.tight_layout()
plt.show()

In [ ]:
print(df.columns)

In [ ]:
#### Building CNN Model with ResNet50 ####

In [ ]:
!pip install tensorflow

In [ ]:
####### Using new normalized, merged image data #####

import os
import re

# Define the directory where your normalized images are stored
image_dir = '/sfs/weka/scratch/axu5pa/DS_4002_Project_3/PH2Dataset/normalized_images'

# Define a regular expression pattern to match your desired file names.
pattern = re.compile(r'^IMD\d{3}_merged\.bmp$')

# List all .bmp files in the directory that match the pattern
image_files = [f for f in os.listdir(image_dir) if f.endswith('.bmp') and pattern.match(f)]

# Create a dictionary to map image names (without .bmp) to file paths
image_dict = {os.path.splitext(f)[0]: os.path.join(image_dir, f) for f in image_files}

# Print the filtered list of image files for verification
print(f"Filtered Image Files: {image_files}")

In [ ]:
##### Updating our data frame #####
import pandas as pd

df['Image Path'] = df['Image Path'].str.replace(r'(?<=/)(.*)(?=\.)', r'\1_merged', regex=True)
print(df['Image Path'].head())

# Check if there are any missing matches
missing_images = df[df['Image Path'].isnull()]

if not missing_images.empty:
    print("Some images were not found:")
    print(missing_images)
else:
    print("All images matched successfully.")

In [ ]:
##### What our dataframe looks like #####
display(df)

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Image Processing Libraries
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# TensorFlow and Keras for Deep Learning
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Scikit-learn for Data Preprocessing and Model Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight  # Import class_weight for computing weights

# Define the directory where your normalized images are stored
image_dir = '/sfs/weka/scratch/axu5pa/DS_4002_Project_3/PH2Dataset/normalized_images'

# Define a regular expression pattern to match 'IMD___' followed by exactly three digits and ending with '.bmp'
pattern = re.compile(r'^IMD\d{3}(_merged)?\.bmp$')

# List all .bmp files in the directory that match the pattern
image_files = [f for f in os.listdir(image_dir) if f.endswith('.bmp') and pattern.match(f)]

# Create a dictionary to map image names (without .bmp) to file paths
image_dict = {os.path.splitext(f)[0]: os.path.join(image_dir, f) for f in image_files}

# Sort the keys in image_dict based on the numerical part of the image names
sorted_image_dict = {k: v for k, v in sorted(image_dict.items(), key=lambda item: int(re.search(r'\d+', item[0]).group()))}

# Map the image paths from the dictionary to the DataFrame (with '_merged' suffix)
df['Image Path'] = df['Image Name'].apply(lambda x: image_dict.get(f'{x}_merged', None))

# Encode the 'Class' labels into integers (Benign = 0, Benign* = 1, Malignant = 2)
label_encoder = LabelEncoder()
df['Encoded Class'] = label_encoder.fit_transform(df['Class'])

# Split the dataset into training and validation sets (80% training, 20% validation)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Encoded Class'], random_state=42)

# Calculate class weights based on the training set
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['Encoded Class']),
    y=train_df['Encoded Class']
)

# Convert class_weights to a dictionary
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Create ImageDataGenerators for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=50,
    zoom_range=0.5,
    shear_range=0.5,
    brightness_range=[0.6, 1.4],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Image Path',
    y_col='Encoded Class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='Image Path',
    y_col='Encoded Class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=False
)

# Load pre-trained ResNet-50 model without top layers (include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers initially for transfer learning
base_model.trainable = False

# Custom top layers added on top of ResNet-50 base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)  # Add Batch Normalization layer to stabilize training

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)           # Dropout layer to prevent overfitting

# Change the number of output units to match the number of classes
predictions = Dense(3, activation='softmax')(x)  # 3 classes for 'Benign', 'Benign*', 'Malignant'

model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze last few layers of ResNet-50 for fine-tuning (experiment with different numbers)
for layer in base_model.layers:
    layer.trainable = True

# Compile model with Adam optimizer and lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks: EarlyStopping and ReduceLROnPlateau (learning rate scheduler)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)

# Train the model with class weights
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // 64,
    validation_data=val_generator,
    validation_steps=max(1, val_generator.n // 64),
    epochs=50,
    class_weight=class_weight_dict,  # Apply class weights here
    callbacks=[early_stopping, reduce_lr_on_plateau],
    verbose=2
)

# Plot training & validation accuracy/loss over epochs
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

# Evaluate final model on validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")



In [ ]:
###### Confusion Matrix ######

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Get true labels from validation dataframe
y_true = val_df['Encoded Class'].values

# Get predictions from model
y_pred = np.argmax(model.predict(val_generator), axis=-1)

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Print explanation
print("""
Confusion Matrix Explanation:
-----------------------------
- Rows represent the actual classes.
- Columns represent the predicted classes.
- Diagonal elements (top-left to bottom-right) represent correct predictions for each class.
- Off-diagonal elements represent misclassifications:
  - Values in row i and column j indicate how many instances of class i were incorrectly predicted as class j.

For example:
- If you see a high value in row 0, column 1, it means many instances of 'Class 0' were incorrectly classified as 'Class 1'.
- Ideally, most of the values should be on the diagonal, indicating correct classifications.
""")

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Image Processing Libraries
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# TensorFlow and Keras for Deep Learning
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Scikit-learn for Data Preprocessing and Model Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight  # Import class_weight for computing weights

# Define the directory where your normalized images are stored
image_dir = '/sfs/weka/scratch/axu5pa/DS_4002_Project_3/PH2Dataset/normalized_images'

# Define a regular expression pattern to match 'IMD___' followed by exactly three digits and ending with '.bmp'
pattern = re.compile(r'^IMD\d{3}(_merged)?\.bmp$')

# List all .bmp files in the directory that match the pattern
image_files = [f for f in os.listdir(image_dir) if f.endswith('.bmp') and pattern.match(f)]

# Create a dictionary to map image names (without .bmp) to file paths
image_dict = {os.path.splitext(f)[0]: os.path.join(image_dir, f) for f in image_files}

# Sort the keys in image_dict based on the numerical part of the image names
sorted_image_dict = {k: v for k, v in sorted(image_dict.items(), key=lambda item: int(re.search(r'\d+', item[0]).group()))}

# Map the image paths from the dictionary to the DataFrame (with '_merged' suffix)
df['Image Path'] = df['Image Name'].apply(lambda x: image_dict.get(f'{x}_merged', None))

# Encode the 'Class' labels into integers (Benign = 0, Benign* = 1, Malignant = 2)
label_encoder = LabelEncoder()
df['Encoded Class'] = label_encoder.fit_transform(df['Class'])

# Split the dataset into training and validation sets (80% training, 20% validation)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Encoded Class'], random_state=42)

# Calculate class weights based on the training set
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['Encoded Class']),
    y=train_df['Encoded Class']
)

# Convert class_weights to a dictionary
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Create ImageDataGenerators for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=50,
    zoom_range=0.5,
    shear_range=0.5,
    brightness_range=[0.6, 1.4],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Image Path',
    y_col='Encoded Class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='Image Path',
    y_col='Encoded Class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=False
)

# Load pre-trained ResNet-50 model without top layers (include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers initially for transfer learning
base_model.trainable = False

# Custom top layers added on top of ResNet-50 base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)  # Add Batch Normalization layer to stabilize training

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)           # Dropout layer to prevent overfitting

# Change the number of output units to match the number of classes
predictions = Dense(3, activation='softmax')(x)  # 3 classes for 'Benign', 'Benign*', 'Malignant'

model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze last few layers of ResNet-50 for fine-tuning (experiment with different numbers)
for layer in base_model.layers:
    layer.trainable = True

# Compile model with Adam optimizer and lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks: EarlyStopping and ReduceLROnPlateau (learning rate scheduler)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)

# Train the model with class weights
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // 64,
    validation_data=val_generator,
    validation_steps=max(1, val_generator.n // 64),
    epochs=50,
    class_weight=class_weight_dict,  # Apply class weights here
    callbacks=[early_stopping, reduce_lr_on_plateau],
    verbose=2
)

# Plot training & validation accuracy/loss over epochs
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

# Evaluate final model on validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


In [ ]:
###### Confusion Matrix ######

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Get true labels from validation dataframe
y_true = val_df['Encoded Class'].values

# Get predictions from model
y_pred = np.argmax(model.predict(val_generator), axis=-1)

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Print explanation
print("""
Confusion Matrix Explanation:
-----------------------------
- Rows represent the actual classes.
- Columns represent the predicted classes.
- Diagonal elements (top-left to bottom-right) represent correct predictions for each class.
- Off-diagonal elements represent misclassifications:
  - Values in row i and column j indicate how many instances of class i were incorrectly predicted as class j.

For example:
- If you see a high value in row 0, column 1, it means many instances of 'Class 0' were incorrectly classified as 'Class 1'.
- Ideally, most of the values should be on the diagonal, indicating correct classifications.
""")

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Image Processing Libraries
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# TensorFlow and Keras for Deep Learning
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Scikit-learn for Data Preprocessing and Model Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define the directory where your normalized images are stored
image_dir = '/sfs/weka/scratch/axu5pa/DS_4002_Project_3/PH2Dataset/normalized_images'

# Define a regular expression pattern to match 'IMD___' followed by exactly three digits and ending with '.bmp'
pattern = re.compile(r'^IMD\d{3}(_merged)?\.bmp$')

# List all .bmp files in the directory that match the pattern
image_files = [f for f in os.listdir(image_dir) if f.endswith('.bmp') and pattern.match(f)]

# Create a dictionary to map image names (without .bmp) to file paths
image_dict = {os.path.splitext(f)[0]: os.path.join(image_dir, f) for f in image_files}

# Sort the keys in image_dict based on the numerical part of the image names
sorted_image_dict = {k: v for k, v in sorted(image_dict.items(), key=lambda item: int(re.search(r'\d+', item[0]).group()))}

# Map the image paths from the dictionary to the DataFrame (with '_merged' suffix)
df['Image Path'] = df['Image Name'].apply(lambda x: image_dict.get(f'{x}_merged', None))

# Encode the 'Class' labels into integers (Benign = 0, Benign* = 1, Malignant = 2)
label_encoder = LabelEncoder()
df['Encoded Class'] = label_encoder.fit_transform(df['Class'])

# Split the dataset into training and validation sets (80% training, 20% validation)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Encoded Class'], random_state=42)

# Create ImageDataGenerators for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=50,
    zoom_range=0.5,
    shear_range=0.5,
    brightness_range=[0.6, 1.4],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create a class-specific generator using the `flow_from_dataframe` method with the balanced classes
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Image Path',
    y_col='Encoded Class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='Image Path',
    y_col='Encoded Class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='raw',
    shuffle=False
)

# Load pre-trained ResNet-50 model without top layers (include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers initially for transfer learning
base_model.trainable = False

# Custom top layers added on top of ResNet-50 base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)  # Add Batch Normalization layer to stabilize training

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)           # Dropout layer to prevent overfitting

# Change the number of output units to match the number of classes
predictions = Dense(3, activation='softmax')(x)  # 3 classes for 'Benign', 'Benign*', 'Malignant'

model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze last few layers of ResNet-50 for fine-tuning (experiment with different numbers)
for layer in base_model.layers:
    layer.trainable = True

# Compile model with Adam optimizer and lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks: EarlyStopping and ReduceLROnPlateau (learning rate scheduler)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // 64,
    validation_data=val_generator,
    validation_steps=max(1, val_generator.n // 64),
    epochs=50,
    callbacks=[early_stopping, reduce_lr_on_plateau],
    verbose=2
)

# Plot training & validation accuracy/loss over epochs
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

# Evaluate final model on validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


In [ ]:
###### Confusion Matrix ######

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Get true labels from validation dataframe
y_true = val_df['Encoded Class'].values

# Get predictions from model
y_pred = np.argmax(model.predict(val_generator), axis=-1)

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Print explanation
print("""
Confusion Matrix Explanation:
-----------------------------
- Rows represent the actual classes.
- Columns represent the predicted classes.
- Diagonal elements (top-left to bottom-right) represent correct predictions for each class.
- Off-diagonal elements represent misclassifications:
  - Values in row i and column j indicate how many instances of class i were incorrectly predicted as class j.

For example:
- If you see a high value in row 0, column 1, it means many instances of 'Class 0' were incorrectly classified as 'Class 1'.
- Ideally, most of the values should be on the diagonal, indicating correct classifications.
""")